##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Crie um pipeline TFX usando templates

Observação: recomendamos executar este tutorial no Google Cloud Vertex AI Workbench. [Iniciar este notebook no Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Ftemplate.ipynb).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

## Introdução

Este documento fornecerá instruções para criar um pipeline TensorFlow Extended (TFX) usando *templates* fornecidos com o pacote TFX Python. Muitas das instruções são comandos shell do Linux que rodam numa instância do AI Platform Notebooks. Células de código correspondentes do Jupyter Notebook que invocam esses comandos usando `!` são fornecidos.

Você construirá um pipeline usando o [dataset Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) disponibilizado pela cidade de Chicago. Recomendamos fortemente que você tente construir seu pipeline usando seu próprio dataset, utilizando esse pipeline como referência.


## Etapa 1. Configure seu ambiente.

O AI Platform Pipelines preparará um ambiente de desenvolvimento para construir um pipeline e um cluster Kubeflow Pipeline para executar o pipeline recém-construído.

**OBSERVAÇÃO:** para selecionar uma versão específica do TensorFlow ou selecionar uma instância de GPU, crie uma instância pré-instalada do TensorFlow nos AI Platform Notebooks.


Instale o pacote `tfx` python com o requisito `kfp` extra.

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

Vamos verificar as versões do TFX.

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

No AI Platform Pipelines, o TFX é executado num ambiente hospedado do Kubernetes usando [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/).

Vamos definir algumas variáveis ​​de ambiente para usar o Kubeflow Pipelines.

Primeiro, obtenha o ID do seu projeto GCP.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

Também precisamos acessar seu cluster KFP. Você pode acessá-lo em seu Console do Google Cloud no menu "AI Platform &gt; Pipeline". O "endpoint" do cluster KFP pode ser encontrado na URL do painel Pipelines ou você pode obtê-lo na URL da página de introdução onde você iniciou este notebook. Vamos criar uma variável de ambiente `ENDPOINT` e configurá-la para o endpoint do cluster KFP. **ENDPOINT deve conter apenas a parte do nome do host da URL.** Por exemplo, se a URL do painel KFP for `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`, o valor ENDPOINT será `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

> **OBSERVAÇÃO: Você PRECISA definir o valor do seu ENDPOINT abaixo.**

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

Defina o nome da imagem como `tfx-pipeline` no projeto GCP atual.

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

E está feito. Estamos prontos para criar um pipeline.

## Etapa 2. Copie o template predefinido para o diretório do seu projeto.

Nesta etapa, criaremos um diretório e arquivos de projeto de pipeline funcionais, copiando arquivos adicionais de um template predefinido.

Você pode dar um nome diferente ao seu pipeline alterando `PIPELINE_NAME` abaixo. Este também se tornará o nome do diretório do projeto onde seus arquivos serão colocados.

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

O TFX inclui o [template <code>taxi</code>](https://github.com/tensorflow/tfx/tree/master/tfx/experimental/templates/taxi) com o pacote TFX Python. Se você está planejando resolver um problema de previsão pontual, incluindo classificação e regressão, este template pode ser usado como ponto de partida.

O comando CLI `tfx template copy` copia arquivos de template predefinidos no diretório do projeto.

In [ ]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

Altere o contexto do diretório de trabalho neste notebook para o diretório do projeto.

In [ ]:
%cd {PROJECT_DIR}

> OBSERVAÇÃO: Não se esqueça de alterar o diretório no `File Browser` à esquerda clicando no diretório do projeto assim que ele for criado.

## Etapa 3. Navegue pelos arquivos-fonte copiados.

O template TFX fornece arquivos de estrutura básicos para construir um pipeline, incluindo código-fonte Python, dados de amostra e Jupyter Notebooks para analisar a saída do pipeline. O modelo `taxi` usa o mesmo dataset do *Chicago Taxi* e modelo de ML do [Tutorial Airflow](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop).

Aqui está uma breve introdução a cada um dos arquivos Python.

- `pipeline` - este diretório contém a definição do pipeline
    - `configs.py` — define constantes comuns para executores de pipeline
    - `pipeline.py` — define componentes TFX e um pipeline
- `models` - este diretório contém definições de modelo de ML.
    - `features.py`, `features_test.py` — define características para o modelo
    - `preprocessing.py`, `preprocessing_test.py` — define jobs de pré-processamento usando `tf::Transform`
    - `estimator` — este diretório contém um modelo baseado em Estimator.
        - `constants.py` — define constantes do modelo
        - `model.py`, `model_test.py` — define o modelo DNN usando o estimador TF
    - `keras` — este diretório contém um modelo baseado em Keras.
        - `constants.py` — define constantes do modelo
        - `model.py`, `model_test.py` — define o modelo DNN usando Keras
- `local_runner.py`, `kubeflow_runner.py` — define executores para cada mecanismo de orquestração


Você poderá perceber que existem alguns arquivos com `_test.py` em seus nomes. Estes são testes unitários do pipeline e é recomendado adicionar mais testes unitários à medida que você for implementando seus próprios pipelines. Você pode executar testes de unidade fornecendo o nome do módulo dos arquivos de teste com o sinalizador `-m`. Geralmente você pode obter um nome de módulo excluindo a extensão `.py` e substituindo `/` por `.`. Por exemplo:

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## Etapa 4. Execute seu primeiro pipeline TFX

Os componentes no pipeline TFX gerarão saídas para cada execução como [artefatos do ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) e precisam ser armazenados em algum lugar. Você pode usar qualquer armazenamento que o cluster KFP possa acessar e, para este exemplo, usaremos o Google Cloud Storage (GCS). Um bucket padrão do GCS deverá ter sido criado automaticamente. Seu nome será `<your-project-id>-kubeflowpipelines-default`.


Vamos fazer upload de nossos dados de amostra para o bucket do GCS para que possamos usá-los em nosso pipeline mais tarde.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

Vamos criar um pipeline TFX usando o comando `tfx pipeline create`.

> Observação: ao criar um pipeline para KFP, precisamos de uma imagem de container que será usada para executar nosso pipeline. E o `skaffold` vai criar a imagem para nós. Como o skaffold extrai imagens base do docker hub, levará de 5 a 10 minutos quando criarmos a imagem pela primeira vez, mas levará muito menos tempo na segunda vez.

In [ ]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

Ao criar um pipeline, um `Dockerfile` será gerado para construir uma imagem Docker. Não esqueça de adicioná-lo ao sistema de controle de fontes (por exemplo, git) junto com outros arquivos-fonte.

OBSERVAÇÃO: o `kubeflow` será automaticamente selecionado como mecanismo de orquestração se o `airflow` não estiver instalado e `--engine` não for especificado.

Agora inicie uma execução com o pipeline recém-criado usando o comando `tfx run create`.

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

Ou você também pode executar o pipeline no painel do KFP. A nova execução será listada em Experiments no Painel do KFP. Clicar no experimento permitirá monitorar o progresso e visualizar os artefatos criados durante a execução.

No entanto, recomendamos acessar o Painel do KFP. Você pode acessar o painel do KFP no menu Cloud AI Platform Pipelines no Google Cloud Console. Depois de acessar o painel, você encontrará o pipeline e poderá acessar diversas informações sobre ele. Por exemplo, você pode encontrar suas execuções no menu *Experiments* e, ao abrir sua execução em Experiments, poderá encontrar todos os seus artefatos no pipeline no menu *Artifacts*.

> Observação: se a execução do pipeline falhar, você poderá ver logs detalhados de cada componente TFX na aba Experiments no painel do KFP.

Uma das principais fontes de falha são problemas relacionados a permissões. Certifique-se de que seu cluster KFP tenha as permissões necessárias para acessar as APIs do Google Cloud. Isto pode ser configurado [quando você cria um cluster KFP no GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up) ou veja o [documento de solução de problemas no GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

## Etapa 5. Adicione componentes para validação de dados.

Nesta etapa, você adicionará componentes para validação de dados, incluindo `StatisticsGen`, `SchemaGen` e `ExampleValidator`. Se você estiver interessado na validação de dados, consulte [Introdução ao Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

> **Dê um duplo-clique para alterar o diretório para `pipeline` e dê outro duplo-clique para abrir `pipeline.py`**. Encontre e descomente as 3 linhas que adicionam `StatisticsGen`, `SchemaGen` e `ExampleValidator` ao pipeline. (Dica: procure comentários contendo `TODO(step 5):`). Certifique-se de salvar `pipeline.py` depois de editá-lo.

Agora você precisa atualizar o pipeline existente com a definição de pipeline modificada. Use o comando `tfx pipeline update` para atualizar seu pipeline, seguido pelo comando `tfx run create` para criar uma nova execução de seu pipeline atualizado.


In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### Verifique as saídas do pipeline

Acesse o painel do KFP para encontrar as saídas do pipeline na página da execução do seu pipeline. Clique na aba *Experiments* à esquerda e *All runs* na página Experiments. Você deverá conseguir encontrar a execução mais recente com o nome do seu pipeline.

## Etapa 6. Adicione componentes para treinamento.

Nesta etapa, você adicionará componentes para treinamento e validação de modelo, incluindo `Transform`, `Trainer`, `Resolver`, `Evaluator` e `Pusher`.

> **Dê um duplo-clique para abrir `pipeline.py`**. Encontre e descomente as 5 linhas que adicionam `Transform`, `Trainer`, `Resolver`, `Evaluator` e `Pusher` ao pipeline. (Dica: pesquise `TODO(step 6):`)

Como fez antes, agora você precisa atualizar o pipeline existente com a definição de pipeline modificada. As instruções são as mesmas da Etapa 5. Atualize o pipeline usando `tfx pipeline update` e crie uma execução usando `tfx run create`.


In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Quando esta execução for concluída com sucesso, você terá criado e executado seu primeiro pipeline TFX usando AI Platform Pipelines!

**OBSERVAÇÃO:** Se alterarmos alguma coisa no código do modelo, também teremos que recompilar a imagem do container. Podemos acionar o rebuild usando o sinalizador `--build-image` no comando `pipeline update`.

**OBSERVAÇÃO:** Você deve ter percebido que toda vez que criamos uma execução de pipeline, cada componente é executado repetidamente, mesmo que a entrada e os parâmetros não tenham sido alterados. É uma perda de tempo e recursos, e você pode pular essas execuções com o cache do pipeline. Você pode ativar o cache especificando `enable_cache=True` para o objeto `Pipeline` em `pipeline.py`.


## Etapa 7. (*Opcional*) Experimente BigQueryExampleGen.

O [BigQuery ](https://cloud.google.com/bigquery) é um data warehouse em nuvem sem servidor, altamente escalonável e econômico. O BigQuery pode ser usado como fonte para exemplos de treinamento no TFX. Nesta etapa, adicionaremos <code>BigQueryExampleGen</code> ao pipeline.

> **Dê um duplo-clique para abrir `pipeline/pipeline.py`**. Comente `CsvExampleGen` e descomente a linha que cria uma instância de `BigQueryExampleGen`. Você também precisa descomentar o argumento `query` da função `create_pipeline`.

Precisamos especificar qual projeto GCP usar para o BigQuery, e isso é feito definindo `--project` em `beam_pipeline_args` ao criar um pipeline.

> **Dê um duplo-clique para abrir `configs.py`**. Descomente a definição de `GOOGLE_CLOUD_REGION`, `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` e `BIG_QUERY_QUERY`. Você deve substituir o valor da região neste arquivo pelos valores corretos para o seu projeto GCP.

> **Observação: você PRECISA definir sua região GCP no arquivo `configs.py` antes de continuar.**

> **Mude o diretório para um nível acima.** Clique no nome do diretório acima da lista de arquivos. O nome do diretório é o nome do pipeline que é `my_pipeline` se você não mudou.

> **Dê um duplo-clique para abrir `kubeflow_runner.py`**. Descomente os dois argumentos, `query` e `beam_pipeline_args`, para a função `create_pipeline`.

Agora o pipeline está pronto para usar o BigQuery como fonte de exemplos. Atualize o pipeline e crie uma nova execução como fizemos nas etapas 5 e 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

## Etapa 8. (*Opcional*) Experimente o Dataflow com o KFP

Vários [componentes TFX usam o Apache Beam](https://www.tensorflow.org/tfx/guide/beam) para implementar pipelines paralelos de dados, o que significa que você poderá distribuir cargas de trabalho de processamento de dados usando o [Google Cloud Dataflow](https://cloud.google.com/dataflow/). Nesta etapa, configuraremos o orquestrador Kubeflow para usar o dataflow como back-end de processamento de dados para o Apache Beam.

> **Dê um duplo-clique em `pipeline` para alterar o diretório e dê outro duplo-clique para abrir `configs.py`**. Descomente a definição de `GOOGLE_CLOUD_REGION` e `DATAFLOW_BEAM_PIPELINE_ARGS`.

> **Mude o diretório para um nível acima.** Clique no nome do diretório acima da lista de arquivos. O nome do diretório é o nome do pipeline que é `my_pipeline` se você não mudou.

> **Dê um duplo-clique para abrir `kubeflow_runner.py`**. Descomente `beam_pipeline_args`. (Certifique-se também de comentar o `beam_pipeline_args` atual que você adicionou na Etapa 7.)

Agora o pipeline está pronto para usar o Dataflow. Atualize o pipeline e crie uma execução como fizemos nas etapas 5 e 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Você pode encontrar seus jobs de Dataflow no [Dataflow no Cloud Console](http://console.cloud.google.com/dataflow).


## Etapa 9. (*Opcional*) Experimente o Cloud AI Platform Training and Prediction com o KFP

O TFX interopera com vários serviços gerenciados do GCP, como o [Cloud AI Platform for Training and Prediction](https://cloud.google.com/ai-platform/). Você pode configurar seu componente `Trainer` para usar o Cloud AI Platform Training, um serviço gerenciado para treinamento de modelos de ML. Além disso, quando seu modelo estiver criado e pronto para ser servido, você poderá *enviá-lo* para o Cloud AI Platform Prediction para disponibilizar o serviço. Nesta etapa, configuraremos nosso componente `Trainer` e `Pusher` para usar os serviços Cloud AI Platform.

> Antes de editar arquivos, talvez seja necessário ativar a *API AI Platform Training &amp; Prediction*.

> **Dê um duplo-clique em `pipeline` para alterar o diretório e dê outro duplo-clique para abrir `configs.py`**. Descomente a definição de `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS` e `GCP_AI_PLATFORM_SERVING_ARGS`. Usaremos nossa imagem de container personalizada para treinar um modelo no Cloud AI Platform Training, portanto, precisamos definir `masterConfig.imageUri` em `GCP_AI_PLATFORM_TRAINING_ARGS` com o mesmo valor de `CUSTOM_TFX_IMAGE` acima.

> **Mude o diretório um nível acima e dê um duplo-clique para abrir `kubeflow_runner.py`**. Descomente `ai_platform_training_args` e `ai_platform_serving_args`.

Atualize o pipeline e crie uma execução como fizemos nas etapas 5 e 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Você pode encontrar seus jobs de treinamento em [Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs). Se o pipeline for concluído com sucesso, você encontrará seu modelo em [Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

## Etapa 10. Ingira os SEUS dados no pipeline

Fizemos um pipeline para um modelo usando o dataset Chicago Taxi. Agora é hora de colocar seus dados no pipeline.

Seus dados podem ser armazenados em qualquer lugar que seu pipeline possa acessar, incluindo GCS ou BigQuery. Você precisará modificar a definição do pipeline para acessar seus dados.

1. Se seus dados estiverem armazenados em arquivos, modifique `DATA_PATH` em `kubeflow_runner.py` ou `local_runner.py` e defina-o como o local de seus arquivos. Se seus dados estiverem armazenados no BigQuery, modifique `BIG_QUERY_QUERY` em `pipeline/configs.py` para consultar seus dados corretamente.
2. Adicione características em `models/features.py`.
3. Modifique `models/preprocessing.py` para [transformar os dados de entrada para treinamento](https://www.tensorflow.org/tfx/guide/transform).
4. Modifique `models/keras/model.py` e `models/keras/constants.py` para [descrever seu modelo de ML](https://www.tensorflow.org/tfx/guide/trainer).
    - Você também pode usar um modelo baseado em estimador. Altere a constante `RUN_FN` para `models.estimator.model.run_fn` em `pipeline/configs.py`.

Consulte o [Guia do componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para mais informações.

## Limpeza

Para limpar todos os recursos do Google Cloud usados ​​neste projeto, [exclua o projeto do Google Cloud](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) usado no tutorial.

Como alternativa, você pode limpar recursos individuais acessando cada console individualmente:

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
